In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
# Dependencies
import requests
import pymongo
import dns
import datetime
from datetime import datetime
from dateutil import parser
import pytz
from django.utils import timezone
from time import sleep
import random
import json
import config
import requests

# Project defined dependencies
import config
from pprint import pprint
# import cl_parser


In [10]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(config.mongo_conn)

In [11]:
# Define database and collection
db = client.getitdone_db
SR_collection = db.sandiego

In [5]:
for SR in SR_collection.find():
    print(SR['service_request_id'])
    break

3190603


In [6]:
SR_collection.find().count()    

<ipython-input-6-3c711ff63af8>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  SR_collection.find().count()


681949

In [7]:
for SR in SR_collection.find():
    print(SR)
    break

{'_id': ObjectId('6062a328659e962620d4dcd6'), 'service_request_id': 3190603, 'date_requested': datetime.datetime(2021, 1, 1, 8, 23), 'case_age_days': 28, 'service_name': 'Container Left Out', 'case_record_type': 'ESD Complaint/Report', 'date_closed': datetime.datetime(2021, 1, 29, 8, 0), 'status': 'Closed', 'lat': 32.749027899999966, 'lng': -117.2405783, 'street_address': '4673 Lotus St', 'council_district': 2, 'comm_plan_code': 23, 'comm_plan_name': 'Ocean Beach', 'case_origin': 'Mobile', 'public_description': '2nd report. Tenant has kept containers out since move in- Nov 1. A warning sticker was placed on container by city approx 3 wks ago. Sticker torn off this week.'}


In [8]:
local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime("2019-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime("2020-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

 
print(dt_start_utc, dt_end_utc)

2019-01-01 08:00:00+00:00 2020-01-01 08:00:00+00:00


In [9]:
filter={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}

result = client['getitdone_db']['sandiego'].find(filter=filter)

In [10]:
i = 0
date_requested_count = {}  
date_closed_count = {}

for SR in result:
    
    if 'date_requested' in SR:            
        date_requested_month = SR['date_requested'].month     
        if (date_requested_month in date_requested_count):
            date_requested_count[date_requested_month] += 1        
        else:
            date_requested_count[date_requested_month] = 1            
    if 'date_closed' in SR:
        date_closed_month = SR['date_closed'].month     
        if (date_closed_month in date_closed_count):
            date_closed_count[date_closed_month] += 1        
        else:
            date_closed_count[date_closed_month] = 1 
    
    # SR['case_age_days']
    # SR['service_name']
    # SR['case_record_type']    
    # SR['status']
    # SR['lat']
    # SR['lng']
    # SR['street_address']
    # SR['council_district']
    # SR['comm_plan_code']
    # SR['comm_plan_name']
    # SR['case_origin']
    # SR['public_description']   

    # i += 1
    # if i > 5:
    #     break

In [11]:
date_requested_count


{1: 19285,
 2: 21736,
 3: 23412,
 4: 23070,
 5: 24885,
 6: 25819,
 7: 33100,
 8: 34470,
 9: 28657,
 10: 27163,
 11: 23375,
 12: 24233}

In [12]:
date_closed_count

{1: 17899,
 7: 31756,
 4: 22835,
 6: 22282,
 2: 17868,
 3: 25208,
 9: 26644,
 8: 33654,
 5: 24256,
 10: 28882,
 12: 22966,
 11: 23247}

In [28]:
def get_sd_api_data(sd_api_url):
    print("import SR from get it done api")        
    response = requests.get(sd_api_url)
    response_json = response.json()  

    count_insert_SR = 0
    count_update_SR = 0

    for SR in response_json:
        public_description = ""
        media_url = ""

        # Get the data from the results
        service_request_id = int(SR["service_request_id"])
        date_requested_string = SR["requested_datetime"] # or any date sting of differing formats.
        date_requested = parser.parse(date_requested_string)
        updated_datetime_string = SR["updated_datetime"] # or any date sting of differing formats.
        updated_datetime = parser.parse(updated_datetime_string)        
        
        if "description" in SR:
            public_description = SR["description"]
        if 'media_url' in SR:
            media_url = SR["media_url"]

        if SR_collection.count_documents({'service_request_id': service_request_id}) == 0:
            print("Insert new service request! ")            
            insert_doc = { 
                "service_request_id": service_request_id,
                "date_requested": date_requested,
                "date_updated": updated_datetime,
                "status": SR["status"],
                "service_code": SR["service_code"],
                "service_name": SR["service_name"],
                "public_description": public_description,
                "street_address": SR["address"],
                "lat": float(SR["lat"]),
                "lng": float(SR["long"]),
                "media_url": media_url
                }
            doc = SR_collection.insert_one(insert_doc)
            count_insert_SR += 1
            # if config.debug:
            #     print(doc)

        else: 
            print(f"SR {service_request_id} already exists, update existing service request!")
            update_doc = SR_collection.find_one_and_update(
                {'service_request_id' : service_request_id},
                {'$set':
                    {
                        "date_requested": date_requested,
                        "date_updated": updated_datetime,
                        "status": SR["status"],
                        "service_code": SR["service_code"],
                        "service_name": SR["service_name"],
                        "public_description": public_description,
                        "street_address": SR["address"],
                        "lat": float(SR["lat"]),
                        "lng": float(SR["long"]),
                        "media_url": media_url
                    }
                    
                },upsert=True
            )
            count_update_SR += 1
            # if config.debug:
            #     print(update_doc)

    return f"{str(count_insert_SR)} SR created. {str(count_update_SR)} SR updated."

            

In [29]:
summary = get_sd_api_data("http://san-diego.spotreporters.com/open311/v2/requests.json")

import SR from get it done api
Insert new service request! 
Insert new service request! 
SR 3287557 already exists, update existing service request!
Insert new service request! 
Insert new service request! 
SR 2819621 already exists, update existing service request!
Insert new service request! 
SR 3307187 already exists, update existing service request!
SR 3309701 already exists, update existing service request!
SR 3309376 already exists, update existing service request!
SR 3300177 already exists, update existing service request!
SR 3309070 already exists, update existing service request!
SR 3304027 already exists, update existing service request!
SR 3309659 already exists, update existing service request!
SR 3309682 already exists, update existing service request!
SR 3304713 already exists, update existing service request!
SR 3309673 already exists, update existing service request!
SR 3309680 already exists, update existing service request!
SR 3309608 already exists, update existing s

In [30]:
print(summary)

5 SR created. 45 SR updated.
